In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
filePaths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filePaths.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import random
from PIL import Image
import matplotlib.pyplot as plt
import pydicom
!conda install gdcm -c conda-forge -y
!pip install pillow
!pip install pylibjpeg
import gdcm
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
regex = '/train/'
trainPaths = []
testPaths = [] 
for item in filePaths:
    match = re.search(regex,item)
    if match:
        trainPaths.append(item)
    else:
        testPaths.append(item)
print(len(trainPaths),len(testPaths))    
detectionDf = pd.read_csv(filePaths[1])
def prepareData(filePath):
    im = pydicom.read_file(filePath)
    data = apply_voi_lut(im.pixel_array,im)
    if im.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    print(data.shape)
    return data

In [ ]:
from matplotlib.patches import Rectangle
import json
path = trainPaths[1906]
print(path)
data = prepareData(path)
img_id = os.path.basename(path)
img_id = img_id.replace('.dcm','_image')
print(img_id)
bbox = detectionDf[detectionDf['id'] == img_id]['boxes']
print(bbox.iloc[0])
plt.figure(1); plt.clf()
plt.imshow(data)
ax = plt.gca()
print()
if type(bbox.iloc[0]) == type(''):
    box_val = eval(bbox.iloc[0])
    for item in box_val:
        x = item.get("x")
        y = item.get("y")
        w = item.get("width")
        h = item.get("height")
        rect = Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
plt.pause(1)